In [1]:
import catboost
import gc
import joblib
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../")
import time
import warnings
warnings.simplefilter("ignore")
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, fbeta_score, make_scorer
from tqdm import tqdm

In [2]:
from utils.common import (
    sigmoid, 
    pad_column_name
)
from utils.constants import *
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric,
    TreeExperiment
)
from utils.eda_helpers import (
    plot_missing_proportion_barchart, 
    get_cols, plot_heatmap
)
from utils.extraction_helpers import read_file
from utils.feature_group import (
    CATEGORY_COLUMNS, CONTINUOUS_COLUMNS, NON_FEATURE_COLUMNS
)

In [3]:
%load_ext autoreload
%autoreload

In [4]:
%%time
labels = pd.read_csv(f"{RAW_DATA_PATH}/train_labels.csv")
# if "target" not in train.columns:
#     train = train.merge(labels, on="customer_ID", how="left")

CPU times: user 257 ms, sys: 26.8 ms, total: 284 ms
Wall time: 284 ms


### Read Model

In [5]:
%%time
lgbm_dart = TreeExperiment(
    exp_full_path="../experiments/5.lgbm_dart_923_half_fix",
    seed=923
)

CPU times: user 2.86 s, sys: 1.12 s, total: 3.98 s
Wall time: 784 ms


In [6]:
# lgbm_dart.feature_imp_summary.to_csv("../experiments/5.lgbm_dart_923_half_fix/feature_importance.csv", index=False)

### Training Inference

In [12]:
%%time
train_agg = read_file(f"{PROCESSED_DATA_PATH}/v3/train_agg.pkl")

Shape of data: (458913, 4733)
CPU times: user 3.96 s, sys: 11.4 s, total: 15.4 s
Wall time: 21.2 s


In [13]:
%%time
train_scores_df = lgbm_dart.inference_full(train_agg)

Model 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [03:07<00:00,  2.04s/it]


Model 2


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [03:06<00:00,  2.02s/it]


Model 3


...

Model 4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [03:12<00:00,  2.09s/it]


Model 5


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [03:21<00:00,  2.19s/it]

CPU times: user 2h 5min 26s, sys: 2min 57s, total: 2h 8min 23s
Wall time: 16min 1s


In [15]:
kf = StratifiedKFold(n_splits=5, random_state=lgbm_dart.seed, shuffle=True)

In [23]:
val_indices = [idx_va for idx_tr, idx_va in kf.split(train_agg, labels["target"])]

In [26]:
for i in range(5):
    train_scores_df.loc[val_indices[i], "cv_score"] = train_scores_df.loc[val_indices[i], f"score{i+1}"].values

In [30]:
train_scores_df.insert(5, "target", labels["target"])

In [23]:
# train_scores_df.to_csv("../experiments/5.lgbm_dart_923_half_fix/train_scores_df.csv", index=False)

In [25]:
train_scores_df = read_file("../experiments/5.lgbm_dart_923_half_fix/train_scores_df.csv")

Shape of data: (458913, 8)


In [26]:
amex_metric_np(train_scores_df["cv_score"], labels["target"])

(0.7971742806455548, 0.9257687652834348, 0.6685797960076749)

In [98]:
del train_agg

### Test Inference

In [7]:
%%time
test_agg1 = read_file(f"{PROCESSED_DATA_PATH}/v3/test_agg1.pkl")
test_agg2 = read_file(f"{PROCESSED_DATA_PATH}/v3/test_agg2.pkl")

Shape of data: (462379, 4732)
Shape of data: (462242, 4732)
CPU times: user 6.43 s, sys: 14 s, total: 20.5 s
Wall time: 28.1 s


In [13]:
%%time
test_agg1 = test_agg1.reset_index()
test_agg2 = test_agg2.reset_index()

In [14]:
test_scores1 = lgbm_dart.inference_full(test_agg1)
test_scores2 = lgbm_dart.inference_full(test_agg2)

Model 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [03:19<00:00,  2.14s/it]


Model 2


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [03:08<00:00,  2.02s/it]


Model 3


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [03:01<00:00,  1.95s/it]


Model 4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [03:17<00:00,  2.12s/it]


Model 5


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [03:17<00:00,  2.12s/it]


In [17]:
test_scores1.shape

(462379, 5)

In [18]:
test_scores2.shape

(462242, 5)

In [20]:
test_scores = pd.concat([test_scores1, test_scores2], ignore_index=True)

In [29]:
# test_scores.to_csv(f"../experiments/5.lgbm_dart_923_half_fix/scores_df_.csv", index=False)

In [33]:
sub = read_file(f"{SUBMISSION_DATA_PATH}/empty.csv")
sub["prediction"] = test_scores.mean(axis=1)

Shape of data: (924621, 1)


In [34]:
sub.to_csv(f"{SUBMISSION_DATA_PATH}/submission_11TH_seed923_4000_features_messy_darts.csv", index=False)

In [ ]:
fi.loc[(fi["last"] - fi["third_last"]) < -15, ["base_feature", "last", "second_last", "third_last"]]

In [ ]:
fi2.loc[(fi2["last"] - fi2["first"]) < -5, ["base_feature", "last", "second_last", "third_last", "first"]]

In [ ]:
plot_heatmap(fi.corr(), annot=True, figsize=(19, 17), fmt=".2f")

In [ ]:
# %%time
# lgbm_gbdt_shuffled = LGBM(
#     exp_full_path="../experiments/lgbm_gbdt_shuffled_exp"
# )
# gbdt_shuffled_master_feature_set = lgbm_gbdt_shuffled.master_feature_set

In [ ]:
# master_feature_set = sorted(gbdt_master_feature_set.union(
#     gbdt_shuffled_master_feature_set
# ))
# len(set(master_feature_set))

### Read Train Data (Error Analysis)

In [ ]:
labels = read_file(f"{RAW_DATA_PATH}/train_labels.csv")

In [ ]:
target = labels["target"]

In [ ]:
%%time
raw_train = read_file(f"{RAW_TRAIN_PICKLE_PATH}/raw_train_data.pkl")

In [ ]:
%%time
train_agg = train_agg.loc[:, list(gbdt_master_feature_set)]

In [ ]:
%%time
train_agg = lgbm_gbdt.inference_fold(train_agg, target)

In [ ]:
train_agg.head()

In [ ]:
%%time
eval_df = get_final_metric_df(train_agg.iloc[:, :-1], target, train_agg["prediction"])

In [ ]:
eval_df.loc[eval_df["is_cutoff"] == 1, "target"].sum() / eval_df["target"].sum()

In [ ]:
%%time
false_negative_indices = eval_df.loc[(eval_df["target"] == 1) & (eval_df["is_cutoff"] == 0)].index.tolist()
false_positive_indices = eval_df.loc[(eval_df["target"] == 0) & (eval_df["is_cutoff"] == 1)].index.tolist()

In [ ]:
len(false_negative_indices), len(false_positive_indices)

In [ ]:
wrong_indices = eval_df.loc[false_negative_indices + false_positive_indices, :].index.tolist()

In [ ]:
len(wrong_indices)

In [ ]:
%%time
summary_count_df = raw_train.groupby("customer_ID").count()
summary_count_df = summary_count_df.reset_index()

In [ ]:
for col in tqdm(summary_count_df.columns[4:]):
    summary_count_df[col] = 1 - (summary_count_df[col] / summary_count_df["row_number"])

In [ ]:
wrong_pred_missing_count_df = summary_count_df.loc[wrong_indices]
wrong_pred_missing_count_df = wrong_pred_missing_count_df.sort_index()
correct_pred_missing_count_df = summary_count_df.loc[~summary_count_df.index.isin(wrong_indices)]

In [ ]:
correct_mean_df = correct_pred_missing_count_df.iloc[:, 3:].mean()

In [ ]:
wrong_mean_df = wrong_pred_missing_count_df.iloc[:, 3:].mean()

In [ ]:
((wrong_mean_df / correct_mean_df - 1).sort_values(ascending=False) > 0).sum()

In [ ]:
((wrong_mean_df / correct_mean_df - 1).sort_values(ascending=False) < 0).sum()

In [ ]:
(wrong_mean_df / (correct_mean_df + wrong_mean_df)).sort_values(ascending=False).to_csv(
    f"{EXP_PATH}/percent_of_missing_from_wrong.csv"
)

In [ ]:
wrong_mean_df["D_124"], correct_mean_df["D_124"]

In [ ]:
train_, test_ = train_test_split(score_df, test_size=0.2, random_state=1020)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()
logreg.fit(train_.drop(columns="target"), train_["target"])

In [ ]:
y_pred = logreg.predict_proba(test_.drop(columns="target"))[:, 1]

In [ ]:
y_true = test_["target"].values

In [ ]:
amex_metric(y_true, y_pred)

In [ ]:
# del train_agg

In [ ]:
# lgbm_gbdt.run_validation_check(data=train_agg, labels=labels["target"].values)

In [ ]:
lgbm_gbdt.val_metrics

In [ ]:
# train_agg = lgbm_dart.run_validation_check(data=train_agg)

In [ ]:
# lgbm_dart.val_metrics

### Error Analysis

In [ ]:
train_agg.loc[train_agg["target"] == 0, "prediction"].describe()

In [ ]:
train_agg.loc[train_agg["target"] == 1, "prediction"].describe()

### High Pred Score Analysis

In [ ]:
success_pos_df = train_agg.loc[
    (train_agg["target"] == 1) & (train_agg["is_cutoff"] == 1)
]

In [ ]:
fail_neg_df = train_agg.loc[
    (train_agg["target"] == 0) & (train_agg["is_cutoff"] == 1)
]

### Low Pred Score Analysis

In [ ]:
success_neg_df = train_agg.loc[
    (train_agg["target"] == 0) & (train_agg["is_cutoff"] == 0)
]

In [ ]:
fail_pos_df = train_agg.loc[
    (train_agg["target"] == 1) & (train_agg["is_cutoff"] == 0)
]

In [ ]:
success_neg_df["num_statements"].describe()

In [ ]:
fail_pos_df["num_statements"].describe()

In [ ]:
base_feature = "P_2"

In [ ]:
# for col in get_cols(success_neg_df, base_feature):
#     my_dict = {
#         'True Negative': success_neg_df[col].dropna().values, 
#         'False Negative': fail_pos_df[col].dropna().values,
#         'True Positive': success_pos_df[col].dropna().values, 
#     }
#     fig, ax = plt.subplots(figsize=(15, 12))
#     ax.boxplot(my_dict.values())
#     ax.set_xticklabels(my_dict.keys())
#     plt.title(f"{col}")
#     plt.show()

In [ ]:
# success_pos_df.to_pickle(f"{EXP_PATH}/lgbm_gbdt_exp3/success_positive.pkl")
# fail_pos_df.to_pickle(f"{EXP_PATH}/lgbm_gbdt_exp3/failed_positive.pkl")
# success_neg_df.to_pickle(f"{EXP_PATH}/lgbm_gbdt_exp3/success_negative.pkl")
# fail_neg_df.to_pickle(f"{EXP_PATH}/lgbm_gbdt_exp3/failed_negative.pkl")

### Read Test Data

In [ ]:
%%time
if 'train_agg' in locals():
    del train_agg
# test_agg = read_file(f"{PROCESSED_DATA_PATH}/test_agg_complete2.pkl")
test_agg = read_file(f"{PROCESSED_DATA_PATH}/test_agg.pkl")
gc.collect()

In [ ]:
%%time
test_agg = test_agg.loc[:, list(gbdt_master_feature_set)]
test_agg.shape

In [ ]:
# lgbm_gbdt.feature_imp_df.iloc[30:60]

In [ ]:
%%time
scores_list = []
batch_size = 5000
for j, model in enumerate(lgbm_gbdt.models.values()):
    score_list = []
    print(f"Model {j + 1}")
    for i in tqdm(range(int(test_agg.shape[0] / batch_size) + 1)):
        score_list.append(model.predict(
            test_agg.loc[int(i*batch_size): int((i+1)*batch_size) - 1, model.feature_name()], 
            raw_score=True
        ))
    scores_list.append(np.concatenate(score_list))

In [ ]:
with open(f"{lgbm_gbdt.path}/best_scores.json", "r+") as outfile:
    best_scores_json = json.load(outfile)
np.mean(list(best_scores_json["validation"].values()))

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# score_df = pd.DataFrame(np.array(scores_list).T, columns=range(1, 6))
# minmax_scaler = MinMaxScaler()
# score_df = pd.DataFrame(minmax_scaler.fit_transform(score_df), columns=range(1, 6))
# score_df.describe()
# test_agg["prediction"] = np.mean(score_df, axis=1) 

In [ ]:
test_agg["prediction"] = np.mean(scores_list, axis=0)

In [ ]:
# test_score_df.applymap(sigmoid).mean(axis=1).values # 

In [ ]:
test_score_df = pd.DataFrame(np.array(scores_list).T, columns=[f"score{i}" for i in range(1, 6)])

In [ ]:
submission = pd.read_csv(f"{SUBMISSION_DATA_PATH}/submission25_0.799.csv")
# submission["prediction"] = test_agg["prediction"].values
submission["prediction"] = test_score_df.applymap(sigmoid).mean(axis=1)
submission.to_csv(f"{SUBMISSION_DATA_PATH}/submission6_0.797055_sigmoid_avg.csv", index=False)

In [ ]:
# test_agg = lgbm_dart.get_test_prediction(test_agg)

In [ ]:
# submission["prediction"].apply(sigmoid)

In [ ]:
test_score_df

#### Take the average of all CV models' predictions

In [ ]:
# np.mean(scores_list, axis=0)

In [ ]:
test_agg["prediction"] = np.mean(scores_list, axis=0)

In [ ]:
def get_metric_columns(df):
    df = df.sort_values(by="prediction", ascending=False)
    df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
    four_pct_cutoff = int(0.04 * df['weight'].sum())
    df['weight_cumsum'] = df['weight'].cumsum()
    df["is_cutoff"] = 0
    df.loc[df['weight_cumsum'] <= four_pct_cutoff, "is_cutoff"] = 1
    df = df.reset_index()
    
    df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
    total_pos = (df['target'] * df['weight']).sum()
    df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
    df['lorentz'] = df['cum_pos_found'] / total_pos
    df['gini'] = (df['lorentz'] - df['random']) * df['weight']
    return df

In [ ]:
test_agg["prediction"].describe()

In [ ]:
del scores_list

In [ ]:
test_agg.loc[(test_agg["P_2_last"] < 0.4) & (test_agg["prediction"] < 0), "prediction"] += 4

#### Take a look on CV score

In [ ]:
with open(f"{EXP_FULL_PATH}/best_scores.json", "r+") as outfile:
    best_scores = json.load(outfile)
np.mean(list(best_scores["validation"].values()))

### Manual Adjustment

In [ ]:
test_agg["P_2_last"].describe()

#### Prepare the submission csv, then export out new submission file

In [ ]:
submission = pd.read_csv(f"{SUBMISSION_DATA_PATH}/submission25_0.799.csv")
submission["prediction"] = test_agg["prediction"].values
submission.to_csv(f"{SUBMISSION_DATA_PATH}/submission__.csv", index=False)

### Analyze Feature Importance

#### Read old feature importance summary df

In [ ]:
old_feature_imp_df = pd.read_csv(f"{EXP_FULL_PATH}/feature_imp_summary.csv")

#### Construct new feature importance summary df

In [ ]:
feature_imp_df.loc[feature_imp_df["agg_type"] == ""]

In [ ]:
corr_df = pd.read_csv(f"{PROCESSED_DATA_PATH}/raw_column_correlation/train_corr_df.csv")
base_features = corr_df.columns.tolist()

In [ ]:
old_feature_imp_df

In [ ]:
new_feature_imp_df.to_csv(f"{PROCESSED_DATA_PATH}/feature_imp_summary.csv", index=False)

In [ ]:
# validation_indices = joblib.load(f'{EXP_FULL_PATH}/5_fold_val_indices.pkl')

## Feature Importance

In [ ]:
# fi_master = read_file(f"{EXP_PATH}/feature_imp_general_summary.csv")

In [ ]:
fi_gbdt = lgbm_gbdt2.feature_imp_summary

In [ ]:
# fi_gbdt.to_csv(f"{EXP_PATH}/feature_imp_summary5.csv", index=False)

In [ ]:
fi_dart = lgbm_dart.feature_imp_df

In [ ]:
lgbm_gbdt.feature_imp_summary.to_csv(f"{EXP_PATH}/feature_imp_summary3.csv", index=False)

In [ ]:
lgbm_gbdt_shuffled.feature_imp_df.groupby("agg_type")["average_importance"].sum().sort_values().iloc[-30:]

In [ ]:
set(fi_gbdt["base_feature"]) - set(fi_dart["base_feature"])

In [ ]:
set(fi_dart["base_feature"]) - set(fi_gbdt["base_feature"])

In [ ]:
# lgbm_dart.feature_imp_df

In [ ]:
imp_df = lgbm_gbdt.feature_imp_df.copy()
imp_summary = lgbm_gbdt.feature_imp_summary.copy()

In [ ]:
half_max_imp_df = imp_df.loc[imp_df["average_importance"] > 0.5*imp_df["average_importance"].max()]

In [ ]:
top_base_features = imp_summary.loc[imp_summary["base_feature"].isin(half_max_imp_df["base_feature"].tolist())]

In [ ]:
plot_heatmap(top_base_features.set_index("base_feature"), annot=True, figsize=(18, 18), fmt=".1f")

In [ ]:
lgbm_gbdt.feature_imp_summary.to_csv(f"{EXP_PATH}/feature_imp_summary3.csv", index=False)

In [ ]:
noob_features_df = feature_imp_df["average_importance"].nsmallest(int(feature_imp_df.shape[0] * 0.04))
noob_features = noob_features_df.index.tolist()

In [ ]:
top_features_df = feature_imp_df["average_importance"].nlargest(int(feature_imp_df.shape[0] * 0.9))
top_features = top_features_df.index.tolist()

In [ ]:
pd.Series([f.split("_")[-1] for f in top_features]).value_counts()

In [ ]:
pd.Series(top_features, name="feature").to_csv("top_features.csv", index=False)

In [ ]:
# used_features = set(feature_imp_df.index.tolist())

In [ ]:
mean_features = pad_column_name(MEAN_FEATURES, suffix="_avg")
min_features = pad_column_name(MIN_FEATURES, suffix="_min")
max_features = pad_column_name(MAX_FEATURES, suffix="_max")
last_features = pad_column_name(LAST_FEATURES, suffix="_last")
first_features = pad_column_name(FIRST_FEATURES, suffix="_first")

In [ ]:
range_features = pad_column_name(RANGE_FEATURES, suffix="_range")
velocity_features = pad_column_name(VELOCITY_FEATURES, suffix="_velocity")
speed_features = pad_column_name(SPEED_FEATURES, suffix="_speed")

In [ ]:
all_features = (
    mean_features + min_features + max_features + last_features + first_features
    + range_features + velocity_features + speed_features
)

In [ ]:
len(all_features)

In [ ]:
set(top_features) - set(all_features)

In [ ]:
top_features

In [ ]:
feature_imp_df = feature_imp_df.loc[feature_imp_df["average_importance"] > 0]
feature_imp_subset = feature_imp_df.loc[feature_imp_df["average_importance"] > 200].reset_index()

In [ ]:
# feature_imp_subset.loc[feature_imp_subset["feature"].str.contains("last")].sort_values("feature")

In [ ]:
t = feature_imp_subset.copy()

In [ ]:
t.loc[t["feature"] == "D_128_first"]

In [ ]:
# feature_imp_df.index.tolist()

In [ ]:
feature_list = feature_imp_df.index.tolist()
len(feature_list)

In [ ]:
original_score

In [ ]:
simulated_score_list = []
for feature in feature_list[::-1]:
    temp_train_val = train_val.copy()
    temp_train_val[feature] = np.nan
    for i, indices in enumerate(validation_indices):
        temp_model = models[i]
        temp_val = temp_train_val.loc[indices, temp_model.feature_name_]
        temp_train_val.loc[indices, "score"] = temp_model.predict_proba(temp_val, raw_score=True)
    simulated_score = amex_metric(temp_train_val["target"], temp_train_val["score"])
    simulated_score_list.append(simulated_score)
    if simulated_score >= original_score:
        sign = "+"
    else:
        sign = ""
    if abs(simulated_score - original_score) >= 1e-4:
        print(feature, f" {sign}{(simulated_score - original_score):.4f}")
    print(f"{feature} done")
    del temp_train_val
    gc.collect()

In [ ]:
test = read_file(f"{RAW_TEST_PICKLE_PATH}/test_data.pkl")

In [ ]:
test.loc[test["row_number"] > 12]

In [ ]:
test_agg = process_test_data(test)

In [ ]:
test_agg.head()

In [ ]:
del test

In [ ]:
for i, model in enumerate(models.values()):
    model_input = test_agg.loc[:, model.feature_name_]
    test_agg.loc[:, f"score{i}"] = model.predict_proba(model_input, raw_score=True)

In [ ]:
test_agg.head()

In [ ]:
new_result_sub = pd.read_csv(f"{SUBMISSION_DATA_PATH}/submission19_0.796.csv")

In [ ]:
old_result_sub = pd.read_csv(f"{SUBMISSION_DATA_PATH}/submission17_0.796.csv")

In [ ]:
plt.scatter(old_result_sub["prediction"], new_result_sub["prediction"])

In [ ]:
np.corrcoef(old_result_sub["prediction"], new_result_sub["prediction"])

In [ ]:
merged_result = new_result_sub.merge(old_result_sub, on="customer_ID", how="left")

In [ ]:
merged_result["x_positive"] = (merged_result["prediction_x"] >= 0)

In [ ]:
merged_result["y_positive"] = (merged_result["prediction_y"] >= 0)

In [ ]:
neg_neg_indices = merged_result.loc[(~merged_result["x_positive"]) & (~merged_result["y_positive"])].index

In [ ]:
merged_result.loc[neg_neg_indices, "prediction"] = merged_result.loc[neg_neg_indices, :].min(axis=1)

In [ ]:
pos_pos_indices = merged_result.loc[(merged_result["x_positive"]) & (merged_result["y_positive"])].index

In [ ]:
merged_result.loc[pos_pos_indices, "prediction"] = merged_result.loc[pos_pos_indices, :].max(axis=1)

In [ ]:
merged_result["prediction"].isnull().sum()

In [ ]:
merged_result.loc[merged_result["prediction"].isnull(), "prediction"] = \
(merged_result.loc[merged_result["prediction"].isnull(), "prediction_x"] + \
merged_result.loc[merged_result["prediction"].isnull(), "prediction_y"]) / 2

In [ ]:
merged_result = merged_result.drop(columns=["prediction_x", "prediction_y", "x_positive", "y_positive"])

In [ ]:
merged_result.to_csv(f"{SUBMISSION_DATA_PATH}/submission23.csv", index=False)

In [ ]:
new_result_sub["prediction"] = np.max([new_result_sub["prediction"].values, old_result_sub["prediction"].values], axis=0)

In [ ]:
new_result_sub["prediction"] += old_result_sub["prediction"]

In [ ]:
new_result_sub["prediction"] /= 2

In [ ]:
new_result_sub.to_csv(f"{SUBMISSION_DATA_PATH}/submission22.csv", index=False)

In [ ]:
result_sub = test_agg.loc[:, [c for c in test_agg.columns if c.startswith("score")]].mean(axis=1).reset_index()

In [ ]:
result_sub.columns = ["customer_ID", "prediction"]

In [ ]:
result_sub.to_csv("submission20.csv", index=False)

In [ ]:
single_record_index = test_agg.loc[test_agg["num_records"] == 1].index.tolist()

In [ ]:
final_df13 = pd.read_csv(f"{SUBMISSION_DATA_PATH}/submission13_0.796.csv")
final_df13["prediction_13"] = sigmoid(final_df13["prediction"])
final_df13 = final_df13.drop(columns="prediction")

In [ ]:
final_df16 = pd.read_csv(f"{SUBMISSION_DATA_PATH}/submission16_0.782.csv")
final_df16["prediction_16"] = sigmoid(final_df16["prediction"])
final_df16 = final_df16.drop(columns="prediction")

In [ ]:
mega_df = final_df13.merge(final_df16, on="customer_ID", how="left")

In [ ]:
mega_df.loc[:, "prediction"] = (
    mega_df.loc[:, "prediction_16"] * 0.1 \
    + mega_df.loc[:, "prediction_13"] * 0.9
)

In [ ]:
mega_df.loc[single_record_index, "prediction"] = (
    mega_df.loc[single_record_index, "prediction_16"] * 0.8 \
    + mega_df.loc[single_record_index, "prediction_13"] * 0.2
)

In [ ]:
mega_df = mega_df.drop(columns=["prediction_13", "prediction_16"])

In [ ]:
mega_df["prediction"] = 0.55 * mega_df["prediction_x"] + 0.45 * mega_df["prediction_y"]
mega_df = mega_df.drop(columns=["prediction_x", "prediction_y"])

In [ ]:
mega_df.to_csv(f"{SUBMISSION_DATA_PATH}/submission18.csv", index=False)